In [36]:
# --- Forget all previous code ---

# --- 1. Define placeholder functions (f0, f1, f2, etc.) ---
def f0():
    """Placeholder function for Test Case 0."""
    print("Executing f0: Test Case 0 logic.")

def f1():
    """Placeholder function for Test Case 1."""
    print("Executing f1: Test Case 1 logic.")

def f2():
    """Placeholder function for Test Case 2."""
    print("Executing f2: Test Case 2 logic.")

# You can define more functions (e.g., f3(), f4()) here if needed.

# --- 2. Create the 'case' mapping (Python dictionary) ---
# This directly represents case[0]=f0, case[1]=f1, etc.
case = {
    0: f0,
    1: f1,
    2: f2,
    # Add more mappings for other functions here (e.g., 3: f3, 4: f4)
}

# --- 3. Define the 'callall()' function ---
def callall():
    """Calls all functions defined in the 'case' mapping."""
    print("\n--- Executing ALL defined test cases ---")
    # Sort keys for consistent execution order, otherwise dictionary order is arbitrary
    for test_id in sorted(case.keys()):
        print(f"Calling case[{test_id}]...")
        case[test_id]() # Call the function associated with the current ID

# --- 4. Implement the main "switch id" logic ---
def switch_id(test_id):
    """
    Acts as a switch-case for test IDs based on your specific logic.
    - If test_id is -1, calls callall().
    - Otherwise, calls the specific function mapped to test_id in the 'case' dictionary.
    """
    print(f"\n--- Input Test ID: {test_id} ---")

    if test_id == -1:
        # Rule: -1 > all
        callall()
    else:
        # Rule: else case[id]()
        # Per your instruction, we assume test_id will always be a valid key if not -1.
        print(f"Calling case[{test_id}]...")
        case[test_id]()

# --- Example Usage ---
print("--- Starting Switch ID Examples ---")

switch_id(0)   # Calls f0()
switch_id(1)   # Calls f1()
switch_id(2)   # Calls f2()
switch_id(-1)  # Calls all functions (f0, f1, f2)

# Per your instruction "no cases of undefined", I will not include calls
# for IDs that are not present in the 'case' dictionary.
# For example, calling switch_id(99) would raise a KeyError if 99 is not in 'case'.

print("\n--- Switch ID Examples Finished ---")

--- Starting Switch ID Examples ---

--- Input Test ID: 0 ---
Calling case[0]...
Executing f0: Test Case 0 logic.

--- Input Test ID: 1 ---
Calling case[1]...
Executing f1: Test Case 1 logic.

--- Input Test ID: 2 ---
Calling case[2]...
Executing f2: Test Case 2 logic.

--- Input Test ID: -1 ---

--- Executing ALL defined test cases ---
Calling case[0]...
Executing f0: Test Case 0 logic.
Calling case[1]...
Executing f1: Test Case 1 logic.
Calling case[2]...
Executing f2: Test Case 2 logic.

--- Switch ID Examples Finished ---


In [1]:
import os
import textwrap
import re

def solve(mapping_input, indent_unit=4, output_path="output.txt"):
    """
    Parses a folder map text, identifies empty folders, builds a folder-only tree,
    and prints/saves the results.
    It includes the emoji-fied original input map and analysis results
    in both console and file output.
    """
    # Determine input source: file path, direct string, or list of lines
    if isinstance(mapping_input, str) and os.path.isfile(mapping_input):
        with open(mapping_input, 'r', encoding='utf-8') as f:
            lines = f.readlines()
    elif isinstance(mapping_input, str):
        lines = mapping_input.splitlines()
    else: # Assume it's already a list of lines
        lines = mapping_input

    # Prepare all output lines in a single list
    out_lines = []

    # --- Phase 1: Add the original input map with emojis to out_lines ---
    out_lines.append("\n--- Original Folder Map (Emoji Display) ---")
    for raw_line in lines:
        stripped_line = raw_line.strip()
        if stripped_line.startswith("Folder map") or stripped_line.startswith("="):
            out_lines.append(stripped_line) # Add headers as is
            continue
        if not stripped_line: # Add blank lines as is
            out_lines.append("")
            continue

        # Use regex to separate the tree structure prefix from the actual content
        match = re.match(r'^([\s│├└─]*)(.*)', raw_line)
        tree_prefix = match.group(1) if match else ''
        content = match.group(2).strip() if match else raw_line.strip()

        emoji_line = tree_prefix # Start with the preserved tree prefix

        # Determine emoji based on whether it's a folder or file
        if content.endswith('/'):
            emoji_line += "📂 " # Use open folder emoji for initial display
        else:
            emoji_line += "📄 " # Use file emoji

        emoji_line += content # Add the content itself
        out_lines.append(emoji_line)
    out_lines.append("-------------------------------------------\n")

    # --- Phase 2: Existing Logic for Analysis and Results ---

    # Helper to strip leading tree markers and spaces for internal processing
    def strip_markers(line):
        return re.sub(r'^[\s│├└─]+', '', line)

    nodes = [] # List to store parsed nodes (folders/files)
    stack = [] # Stack to keep track of current parent folders for indentation
    id_counter = 0 # Unique ID for each node

    # Re-iterate over lines for node parsing (separate from printing for clarity)
    for raw_line in lines:
        # Skip header/separator lines and truly empty lines that aren't part of tree
        if raw_line.startswith("Folder map") or raw_line.startswith("=") or not raw_line.strip():
            continue

        # Calculate indentation level to determine hierarchy
        m = re.match(r'^[\s│]+', raw_line)
        indent_len = len(m.group()) if m else 0
        level = indent_len // indent_unit # Convert indent length to hierarchical level

        name = strip_markers(raw_line).strip() # Get clean name, remove trailing newline
        is_folder = name.endswith('/')

        # Pop from stack until the top node has a smaller level (i.e., is the parent)
        while stack and stack[-1][0] >= level:
            stack.pop()
        parent_id = stack[-1][1] if stack else None # Assign parent_id

        # Add current node to list of nodes
        nodes.append({
            'id': id_counter,
            'name': name,
            'is_folder': is_folder,
            'parent_id': parent_id
        })

        # If it's a folder, push onto stack for future children
        if is_folder:
            stack.append((level, id_counter))
        id_counter += 1

    # --- Build relationships and identify empty folders ---

    # Map node IDs to their children for folders
    children = {n['id']: [] for n in nodes if n['is_folder']}
    for n in nodes:
        pid = n['parent_id']
        if pid in children:
            children[pid].append(n['id'])

    # Identify truly empty folders (folders with no children in the 'children' map)
    empty_folders = {fid for fid, ch in children.items() if not ch}

    # Create a quick lookup map from ID to node object
    id_to_node = {n['id']: n for n in nodes}

    # Helper function to build the full path of a folder
    def build_path(node):
        parts = []
        cur = node
        while cur:
            parts.append(cur['name'].rstrip('/'))
            cur = id_to_node.get(cur['parent_id'])
        return '/'.join(reversed(parts)) + '/'

    # --- Add analysis output lines to out_lines ---
    # Add a blank line for separation before analysis results, if needed
    out_lines.append("") # Ensure separation between original map and analysis results

    out_lines.append("--- Analysis Results (Emoji Display) ---")
    out_lines.append(f"📂 Empty folders ({len(empty_folders)} total):")
    if empty_folders:
        for fid in sorted(empty_folders):
            out_lines.append(f"– 📁 {build_path(id_to_node[fid])}")
    else:
        out_lines.append("No empty folders found.") # Indicate if no empty folders

    out_lines.append("") # Blank line for separation
    out_lines.append("📁 Folder-only tree:")

    # Sort children alphabetically for consistent tree output
    for ch_ids in children.values():
        ch_ids.sort(key=lambda cid: id_to_node[cid]['name'])

    # Recursive function to build tree lines
    def build_tree_lines(node_id, prefix=""):
        node = id_to_node[node_id]
        # Determine emoji and 'empty' mark based on folder status
        is_empty = node_id in empty_folders
        emoji = "📁" if is_empty else "📂" # 📁 for empty, 📂 for non-empty
        mark = " (empty)" if is_empty else ""
        out_lines.append(f"{prefix}{emoji} {node['name']}{mark}")

        # Filter for only folder children to build the folder-only tree
        folder_children = [cid for cid in children.get(node_id, []) if id_to_node[cid]['is_folder']]
        for i, cid in enumerate(folder_children):
            last_child = (i == len(folder_children) - 1)
            # Choose correct branch connector based on if it's the last child
            branch_connector = "└── " if last_child else "├── "
            # Recursively call for children, updating prefix
            build_tree_lines(cid, prefix + ("    " if last_child else "│   ") + branch_connector)

    # Identify root folders (those with no parent) and add their tree lines
    root_folders = sorted([n['id'] for n in nodes if n['parent_id'] is None and n['is_folder']],
                          key=lambda cid: id_to_node[cid]['name'])
    if not root_folders:
        out_lines.append("No root folders found in the map.")
    for root_id in root_folders:
        build_tree_lines(root_id)

    # Add the closing footer for analysis results
    out_lines.append("----------------------------------------")

    # --- Write ALL accumulated output lines to file ---
    with open(output_path, 'w', encoding='utf-8') as f:
        f.write("\n".join(out_lines))ss

    # --- Print ALL accumulated output lines to console ---
    # This ensures both the initial map and analysis are printed together to console.
    print("\n".join(out_lines))ss


if __name__ == "__main__":
    sample_input = textwrap.dedent("""
    Folder map of c:\\Users\\dell\\Downloads\\Project
    ==================================================

    ├── .vscode/
    │   └── metago_bookmarks.json
    ├── # ­ Active Malware or Exploits posing as.md
    ├── %9.js
    ├── 0_today.txt
    ├── my_empty_folder/
    ├── another_folder/
    │   ├── sub_folder_with_file/
    │   │   └── file.txt
    │   ├── empty_sub_folder/
    │   └── sub_folder_with_another_empty/
    │       └── inner_empty/
    """)
    
    # Call solve; it now handles all printing and file writing internally.
    solve(sample_input, indent_unit=4, output_path='tree_output_fixed.txt')

SyntaxError: invalid syntax (2052729717.py, line 168)